In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
NewsPred = pd.read_csv("C:/Users/deepak/Desktop/Git/Project/UI Popularly/train.csv")
NewsPred.head()

,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,Popularity
0,9.0,744.0,0.497970,1.0,0.689655,9.0,0.0,0.0,1.0,4.666667,...,0.100000,0.8,-0.249621,-0.600000,-0.083333,0.900000,0.800000,0.400000,0.800000,1
1,12.0,623.0,0.503268,1.0,0.675603,5.0,4.0,9.0,0.0,4.715891,...,0.100000,1.0,-0.235185,-0.700000,-0.100000,0.100000,0.200000,0.400000,0.200000,1
2,14.0,493.0,0.582485,1.0,0.777778,9.0,0.0,1.0,0.0,4.667343,...,0.100000,1.0,-0.312500,-0.500000,-0.125000,0.727273,0.318182,0.227273,0.318182,0
3,10.0,126.0,0.821138,1.0,0.892857,4.0,3.0,0.0,1.0,5.071429,...,0.121719,0.5,-0.433333,-0.433333,-0.042138,0.650000,0.350000,0.150000,0.350000,1
4,11.0,238.0,0.608696,1.0,0.773050,5.0,3.0,1.0,0.0,4.756303,...,0.100000,0.8,-0.316667,-0.500000,-0.050000,0.066667,0.000000,0.433333,0.000000,1


In [3]:
NewsPred.isnull().sum()

n_tokens_title                   0
n_tokens_content                 0
n_unique_tokens                  0
n_non_stop_words                 0
n_non_stop_unique_tokens         0
num_hrefs                        0
num_self_hrefs                   0
num_imgs                         0
num_videos                       0
average_token_length             0
num_keywords                     0
data_channel_is_lifestyle        0
data_channel_is_entertainment    0
data_channel_is_bus              0
data_channel_is_socmed           0
data_channel_is_tech             0
data_channel_is_world            0
kw_min_min                       0
kw_max_min                       0
kw_avg_min                       0
kw_min_max                       0
kw_max_max                       0
kw_avg_max                       0
kw_min_avg                       0
kw_max_avg                       0
kw_avg_avg                       0
self_reference_min_shares        0
self_reference_max_shares        0
self_reference_avg_s

In [4]:
# Change is_monday, etc to days
merge = NewsPred[["weekday_is_monday", "weekday_is_tuesday", "weekday_is_wednesday",
"weekday_is_thursday", "weekday_is_friday", "weekday_is_saturday",
"weekday_is_sunday"]]

merge.columns

Index(['weekday_is_monday', 'weekday_is_tuesday', 'weekday_is_wednesday',
       'weekday_is_thursday', 'weekday_is_friday', 'weekday_is_saturday',
       'weekday_is_sunday'],
      dtype='object')

In [5]:
def week_encode(msg):
    if msg.endswith('monday'):
        return 1
    elif msg.endswith('tuesday'):
        return 2
    elif msg.endswith('wednesday'):
        return 3
    elif msg.endswith('thursday'):
        return 4
    elif msg.endswith('friday'):
        return 5
    elif msg.endswith('saturday'):
        return 6
    elif msg.endswith('sunday'):
        return 7
    else: 
        return 0

In [6]:
NewsPred['day'] = merge.apply(lambda x: week_encode(np.argmax(x)),axis=1)

# drop various days
for df in [NewsPred]:
    df.drop(columns=merge,inplace=True)

In [7]:
# Change data_channel_is_entertainment, etc to days

channels = NewsPred.loc[:,"data_channel_is_lifestyle":"data_channel_is_world"].columns

def channel_encode(msg):
    if msg.endswith('lifestyle'):
        return 1
    elif msg.endswith('entertainment'):
        return 2
    elif msg.endswith('bus'):
        return 3
    elif msg.endswith('socmed'):
        return 4
    elif msg.endswith('tech'):
        return 5
    elif msg.endswith('world'):
        return 6
    else: 
        return 0

In [8]:
NewsPred['channel'] = merge.apply(lambda x: channel_encode(np.argmax(x)),axis=1)

# drop various channels
for df in [NewsPred]:
    df.drop(columns=channels,inplace=True)

In [9]:
NewsPred.T

,0,1,2,3,4,5,6,7,8,9,...,27740,27741,27742,27743,27744,27745,27746,27747,27748,27749
n_tokens_title,9.000000,12.000000,14.000000,10.000000,11.000000,9.000000,10.000000,10.000000,10.000000,7.000000,...,7.000000,11.000000,13.000000,12.000000,10.000000,8.000000,11.000000,11.000000,7.000000,10.000000
n_tokens_content,744.000000,623.000000,493.000000,126.000000,238.000000,391.000000,331.000000,594.000000,901.000000,604.000000,...,1065.000000,1077.000000,222.000000,662.180391,1012.000000,123.000000,218.000000,172.000000,90.000000,290.000000
n_unique_tokens,0.497970,0.503268,0.582485,0.821138,0.608696,0.547804,0.621951,0.498246,0.434298,0.562607,...,0.429651,0.455315,0.714953,0.397628,0.472946,0.723577,0.624390,0.666667,0.865169,0.680702
n_non_stop_words,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
n_non_stop_unique_tokens,0.689655,0.675603,0.777778,0.892857,0.773050,0.726027,0.813131,0.695652,0.637827,0.774929,...,0.603503,0.648475,0.896825,0.575296,0.659574,0.859155,0.716216,0.776699,0.923077,0.790576
num_hrefs,9.000000,5.000000,9.000000,4.000000,5.000000,8.000000,13.000000,27.000000,8.000000,8.000000,...,6.000000,22.000000,4.000000,16.000000,39.000000,3.000000,14.000000,5.000000,4.000000,9.000000
num_self_hrefs,0.000000,4.000000,0.000000,3.000000,3.000000,3.000000,0.000000,4.000000,8.000000,3.000000,...,6.000000,6.000000,4.000000,10.000000,3.733203,0.000000,2.000000,3.000000,3.000000,4.000000
num_imgs,0.000000,9.000000,1.000000,0.000000,1.000000,0.000000,1.000000,21.000000,16.000000,1.000000,...,20.000000,13.000000,0.000000,3.000000,1.000000,0.000000,10.000000,2.000000,0.000000,1.000000
num_videos,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.645704,...,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2.000000,0.000000
average_token_length,4.666667,4.715891,4.667343,5.071429,4.756303,4.508951,5.054381,4.498316,4.499445,4.302980,...,4.394366,4.468895,4.549550,4.418605,4.775692,4.878049,4.756881,4.691860,4.322222,5.151724


In [11]:
"""
X = NewsPred[['n_tokens_title', 'n_tokens_content', 'n_unique_tokens',
       'n_non_stop_words', 'n_non_stop_unique_tokens', 'num_hrefs',
       'num_self_hrefs', 'num_imgs', 'num_videos', 'average_token_length',
       'num_keywords', 'data_channel_is_lifestyle',
       'data_channel_is_entertainment', 'data_channel_is_bus',
       'data_channel_is_socmed', 'data_channel_is_tech',
       'data_channel_is_world', 'kw_min_min', 'kw_max_min', 'kw_avg_min',
       'kw_min_max', 'kw_max_max', 'kw_avg_max', 'kw_min_avg', 'kw_max_avg',
       'kw_avg_avg', 'self_reference_min_shares', 'self_reference_max_shares',
       'self_reference_avg_sharess', 'weekday_is_monday', 'weekday_is_tuesday',
       'weekday_is_wednesday', 'weekday_is_thursday', 'weekday_is_friday',
       'weekday_is_saturday', 'weekday_is_sunday', 'is_weekend', 'LDA_00',
       'LDA_01', 'LDA_02', 'LDA_03', 'LDA_04', 'global_subjectivity',
       'global_sentiment_polarity', 'global_rate_positive_words',
       'global_rate_negative_words', 'rate_positive_words',
       'rate_negative_words', 'avg_positive_polarity', 'min_positive_polarity',
       'max_positive_polarity', 'avg_negative_polarity',
       'min_negative_polarity', 'max_negative_polarity', 'title_subjectivity',
       'title_sentiment_polarity', 'abs_title_subjectivity',
       'abs_title_sentiment_polarity']]
    
    """
X = NewsPred[["n_tokens_title","n_tokens_content","channel","num_hrefs","num_imgs","num_videos","day"]]
y = NewsPred['Popularity']

In [12]:
X_train,X_test,y_train,y_test=train_test_split(X,y,stratify=y,test_size=0.20,random_state=0)

In [13]:
logreg = LogisticRegression()
logreg.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [14]:
y_pred=logreg.predict(X_test)

In [15]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.5601801801801802


In [16]:
import pickle 
with open('model.pkl','wb') as model_pkl:
    pickle.dump(logreg, model_pkl)